## **数据检索**
用户输入只是问题,答案需要从已知的数据库或者文档中获取。因此,我们需要使用检索器获取上下文,并通过"question"键下的用户输入。
## **RAG**
检索增强生成(Retrieval-augmented Generation,RAG),是当下最热门的大模型前沿技术之一。如果将"微调(finetune)"理解成大横型内为化吸收知识的过
程,那么RAG就相当于给大横型装上了知识外挂",基础大横型不用再训练即可随时调用特定领域知识。

## **文本加载器**
用文本加载器加载知识库给大模型实现检索增强,以下是一段示例代码

In [1]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings

# 初始化文本嵌入模型
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key=os.getenv("API_KEY")
)

# 创建一个FAISS向量数据库
vector_store = FAISS.from_texts(
    texts=["小明今年18岁", "小王今年20岁", "小李今年22岁"],
    embedding=embeddings
)

# 将向量数据库转换为检索器
retriever = vector_store.as_retriever();

# 使用检索器获取上下文
context = retriever.get_relevant_documents("小明今年多少岁？")

print(f"打印检索器获取的相关内容\n{context}")

# 根据上下文调用大模型回答问题
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="qwen-turbo",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL")
)

# 创建一个提示模板
from langchain_core.prompts import ChatPromptTemplate

template = """
你是一个AI助手,请根据以下上下文回答用户的问题。

上下文:
{context}

问题: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# 定义结构化解析器
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

# 定义输出解析器
output_parser = StrOutputParser()

# 方法一
# 定义并行处理
# RunnableParallel 的主要好处是通过并行执行提高效率、简化代码结构。
# 使复杂的数据流程更加清晰和易于管理，特别适合于需要同时处理多个独立任务的 LangChain 应用场景。
# setup_and_retrieval = RunnableParallel(
#     {
#         "context": lambda x: retriever.get_relevant_documents(x["question"]),
#         "question": RunnablePassthrough()
#     }
# )

# 将提示模板和结构化解析器结合起来
# chain = setup_and_retrieval | prompt | llm | output_parser

# 执行链
# chain.invoke({"question": "小明今年多少岁？"})

# 方法二
# 完整的RAG链式结构
rag_chain = (
    {
        # 接收问题并处理
        "question": RunnablePassthrough(),
        # 根据问题检索文档并格式化
        "context": lambda x: retriever.invoke(x["question"])
    }
    # 并行处理上述两个步骤
    | prompt 
    # 将处理好的提示发送给LLM
    | llm 
    # 解析LLM的输出
    | StrOutputParser()
)

# 执行RAG链
output1 = rag_chain.invoke({"question": "小明今年多少岁？"})

print("\n使用RAG链\n",output1)

output2 = llm.invoke("小明今年多少岁？")

print("\n\n使用LLM\n",output2)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_5064\4035098444.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents("小明今年多少岁？")


打印检索器获取的相关内容
[Document(id='fd06c050-46ed-4eac-bae9-abb2023cadf2', metadata={}, page_content='小明今年18岁'), Document(id='e8d0f032-c9ef-4151-9da7-af697784e0bb', metadata={}, page_content='小李今年22岁'), Document(id='4a876cc1-2406-4d17-b939-e58de7936696', metadata={}, page_content='小王今年20岁')]

使用RAG链
 小明今年18岁。


使用LLM
 content='抱歉，我无法回答关于小明年龄的问题。如果你有其他问题或需要帮助，请告诉我！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-turbo', 'system_fingerprint': None, 'id': 'chatcmpl-cf212272-8e47-9964-86b4-51fc61a901ce', 'finish_reason': 'stop', 'logprobs': None} id='run-880fcb5c-0513-4228-9030-ed09774af727-0' usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {}, 'output_token_details': {}}


## **文档加载器**
加载目录中的文件作为知识库给大模型实现检索增强,以下是一段示例代码

In [32]:
# 导入必要的库
import os
# 目录加载器
from langchain_community.document_loaders import DirectoryLoader    
# 文本加载器
from langchain_community.document_loaders import TextLoader
# 向量数据库
from langchain_community.vectorstores import FAISS
# 文本嵌入模型
from langchain_community.embeddings import DashScopeEmbeddings
# 导入文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 初始化加载器,仅加载txt文件
loader = DirectoryLoader(
    "D:\\AI_Project\\LangChainTutorials\\LangChain\\DB", 
    glob="**/*.txt",
    loader_cls=TextLoader
)

# 加载文档
documents = loader.load()

print(f"当前加载的文档信息:\n{documents}")

# 初始化文本分割器，设置合适的块大小和重叠
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,     # 块大小，确保在模型限制内
    chunk_overlap=100,   # 块重叠以保持上下文
    length_function=len,
)

# 分割文档
split_documents = text_splitter.split_documents(documents)

print(f"当前分割后的文档信息:\n{split_documents}")

# 初始化文本嵌入模型
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key=os.getenv("API_KEY")
)

# 创建一个FAISS向量数据库
vector_store = FAISS.from_documents(
    documents= split_documents,
    embedding=embeddings
)

# 将向量数据库转换为检索器
# 1.**最大边际相关性检索(MMR)
# 想象你在一个图书馆里找关于"猫咪"的书籍,XVR就像是一个智能助手,帮你挑选书籍,
# 它会先找到一本最相关的书,比如《猫咪百科全书》,
# 然后,它会寻找第二本书,这本书不仅要与"睡味"相关,
# 还要与第一本书的内容有所不同,比如可能会找一本关于"猫咪训练"的书,
# WIR的目标是让你既能找到有用的信息,又能获得不同方面的知识。
# 2.**余弦相似度**;
# 余弦相似度是一种测量两个向量在多维空间中角度的方法,
# 继续用图书馆的的例子,你可以把每本书想象成一个向量,它的每个维度代表这本书包含的不同关键词,余弦相似度会计算两本书的向量之间的角度,如果两本书的向量角度接近0度,那么它们在内容上非常相似;如果角度接近90度,那么?它们的
# 内容就不那么相似。
# 总结一下:
# MMR是一个帮你挑选书籍的策略,它力求找到既相关又多样的书籍组合。
# 余弦相似度是一个测量两本书内容相似度的工具,它通过比较书中的关键同向量来判断
retriever_mmr = vector_store.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 1}
)

# 余弦相似度检索,设置相似度阈值为0.5
retriever_similarity = vector_store.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs = {"score_threshold" : 0.5}
)

# 使用检索器获取最相关的文档信息
context_mmr = retriever_mmr.get_relevant_documents("如何变得有钱？");

context_similarity = retriever_similarity.get_relevant_documents("如何变得有钱？");

print(f"打印检索器获取的相关内容\n{context_mmr}")

print(f"打印检索器获取的相关内容\n{context_similarity}")

# 根据上下文调用大模型回答问题
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="qwen-turbo",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL")
)

# 创建一个提示模板
from langchain_core.prompts import ChatPromptTemplate

template = """
你是一个AI助手,请根据文档内容回答用户的问题。

文档内容:
{context}

问题: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# 定义结构化解析器
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

# 定义输出解析器
output_parser = StrOutputParser()


# 方法二
# 完整的RAG链式结构
rag_chain = (
    {
        # 接收问题并处理
        "question": RunnablePassthrough(),
        # 根据问题检索文档并格式化
        "context": lambda x: retriever_mmr.invoke(x["question"])
    }
    # 并行处理上述两个步骤
    | prompt 
    # 将处理好的提示发送给LLM
    | llm 
    # 解析LLM的输出
    | StrOutputParser()
)

# 执行RAG链
output1 = rag_chain.invoke({"question": "如何变得有钱？"})

print("\n使用RAG链\n",output1)

output2 = llm.invoke("如何变得有钱？")

print("\n\n使用LLM\n",output2)

当前加载的文档信息:
[Document(metadata={'source': 'D:\\AI_Project\\LangChain\\DB\\book0.txt'}, page_content='The Project Gutenberg eBook of Four square Jane\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: Four square Jane\n\nAuthor: Edgar Wallace\n\nIllustrator: C. Dudley Tennant\n\nRelease date: April 7, 2025 [eBook #75808]\n\nLanguage: English\n\nOriginal publication: London: The Readers Library Publishing Company Ltd, 1929\n\nCredits: an anonymous Project Gutenberg volunteer\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK FOUR SQUARE JANE ***\n\n\n\n\n\n FOUR SQUARE\n JANE\n\n BY\n E

No relevant docs were retrieved using the relevance score threshold 0.5


打印检索器获取的相关内容
[Document(id='1dd2bc3e-15a9-4cea-a5fb-75d4058c4304', metadata={'source': 'D:\\AI_Project\\LangChain\\DB\\book1.txt'}, page_content='可大部分人却只知道一种方法：努力工作、储蓄或者借贷。 \n \n\u3000\u3000那么，为什么你想提高自己的理财能力呢？因为你想成为那种能够为自己创造机遇的人。你希望能坦然地接受发生的任何事情，并努力使事情变得更好。很少有人知道机遇和金钱是可以创造的。但是，如果你想更幸运，挣到更多的钱，而不只是辛苦工作，那么你的理财能力就非常关键。如果你是那种等待“好事情”发生的人，那么，你就可能要等非常长的时间。这就好比在动身旅行之前非要等待前面5英里长的路上所有的红绿灯都变成绿色一样。 \n \n\u3000\u3000小时候，富爸爸经常教导我和迈克：金钱不是真实的资产。 \n \n\u3000\u3000从我们第一次用牙膏皮“造钱”起，富爸爸就常启发我们去了解金钱的秘密。“穷人和中产阶级为金钱而工作，”他说，“富人则创造金钱。当然不是像你们那样‘造’钱。你把金钱看得越重要，你就会为金钱工作得越辛苦。如果你能够懂得‘金钱不是真实的资产’这一道理，你就会更快地富起来。” \n \n\u3000\u3000“那金钱是什么？”我和迈克反问道，“既然金钱不是真实的资产。” \n \n\u3000\u3000“它是我们大家都认可的东西。”富爸爸回答说。 \n \n\u3000\u3000我们惟一的、最重要的资产是我们的头脑。如果受到良好训练，转瞬间它就能创造大量财富，并使财富从此不再只是三百年前国王和王后们的专属。而一个未经训练的头脑通过教给自己的家庭不正确的生活方式，将会延续给后代极度贫困的生活。 \n \n\u3000\u3000在信息时代，金钱越来越变得让人不可思议，一些人仅凭思想和所谓的合约就能从一无所有而一夜暴富。但如果你询问那些以从事股票买卖或投资活动为生的人，这是怎么回事，他们会把这视为稀松平常。因为他们中常有人从一无所有瞬间变成百万富翁，并且这一切只是通过买卖合约而不是进行实际的金钱交易来进行的。它们通常只是交易所的一个手势，或是从里斯本移到多伦多的交易商面前的荧光屏上的一个光点，

## **PDF文档加载器**
加载PDF文件作为知识库给大模型实现检索增强,以下是一段示例代码

In [34]:
# 导入必要的库
import os
# PDF加载器
from langchain_community.document_loaders import PyPDFLoader    
# PDF加载器
from langchain_community.document_loaders import PyPDFDirectoryLoader    
# 向量数据库
from langchain_community.vectorstores import FAISS
# 文本嵌入模型
from langchain_community.embeddings import DashScopeEmbeddings
# 导入文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 初始化PDF文件加载器
file_loader = PyPDFLoader("D:\\AI_Project\\LangChainTutorials\\LangChain\\DB\\富爸爸穷爸爸.pdf")

# 初始化目录加载器,仅加载PDF文件
directory_loader = PyPDFDirectoryLoader(
    "D:\\AI_Project\\LangChainTutorials\\LangChain\\DB", 
    glob="**/*.PDF"
)

# 加载文档
documents = directory_loader.load()

print(f"当前加载的文档信息:\n{documents}")

# 初始化文本分割器，设置合适的块大小和重叠
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,     # 块大小，确保在模型限制内
    chunk_overlap=100,   # 块重叠以保持上下文
    length_function=len,
)

# 分割文档
split_documents = text_splitter.split_documents(documents)

print(f"当前分割后的文档信息:\n{split_documents}")

# 初始化文本嵌入模型
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key=os.getenv("API_KEY")
)

# 创建一个FAISS向量数据库
vector_store = FAISS.from_documents(
    documents= split_documents,
    embedding=embeddings
)

# 余弦相似度检索
retriever_similarity = vector_store.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs = {"score_threshold" : 0.3}
)

# 使用检索器获取最相关的文档信息
context_similarity = retriever_similarity.get_relevant_documents("如何变得有钱？");

print(f"打印检索器获取的相关内容\n{context_similarity}")

# 根据上下文调用大模型回答问题
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="qwen-turbo",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL")
)

# 创建一个提示模板
from langchain_core.prompts import ChatPromptTemplate

template = """
你是一个AI助手,请根据文档内容回答用户的问题。

文档内容:
{context}

问题: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# 定义结构化解析器
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

# 定义输出解析器
output_parser = StrOutputParser()


# 方法二
# 完整的RAG链式结构
rag_chain = (
    {
        # 接收问题并处理
        "question": RunnablePassthrough(),
        # 根据问题检索文档并格式化
        "context": lambda x: retriever_mmr.invoke(x["question"])
    }
    # 并行处理上述两个步骤
    | prompt 
    # 将处理好的提示发送给LLM
    | llm 
    # 解析LLM的输出
    | StrOutputParser()
)

# 执行RAG链
output1 = rag_chain.invoke({"question": "如何变得有钱？"})

print("\n使用RAG链\n",output1)

output2 = llm.invoke("如何变得有钱？")

print("\n\n使用LLM\n",output2)

当前加载的文档信息:
[Document(metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': 'D:20100119151156', 'title': '富爸爸  穷爸爸', 'moddate': 'D:20100119151156', 'source': 'D:\\AI_Project\\LangChain\\DB\\富爸爸穷爸爸.pdf', 'total_pages': 273, 'page': 0, 'page_label': '1'}, page_content='富爸爸  \n穷爸爸 \n \n罗伯特〃T〃清崎 莎伦〃L〃莱希特'), Document(metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': 'D:20100119151156', 'title': '富爸爸  穷爸爸', 'moddate': 'D:20100119151156', 'source': 'D:\\AI_Project\\LangChain\\DB\\富爸爸穷爸爸.pdf', 'total_pages': 273, 'page': 1, 'page_label': '2'}, page_content='目   录 \n书籍介绍: ......................................................................... 3 \n第一部分 课程 ................................................................. 5 \n第1 章 富爸爸，穷爸爸 ................................ ..............  5 \n第2 章 第一课:富人不为钱工作................................  16 \n第3 章 第二课:为什么要教授财务知识 ................